# Query the Neuroimaging and Phenotypic Data (Used in Meisler and Gabrieli, 2021)
## This code contains
1) Phenotypic data loading
2) Quality Control
3) Cohort Statistic Calculations (e.g. Table 1 and Figure 1)
4) Code for TractSeg model creation

### If replicating this study, make sure your dataset is BIDS-compliant and:
1) Update the variable "bids_dir" to reflect where your data is stored
2) Have your phenotypic data stored in the BIDS code directory with the name "HBN_query.csv"
3) Make sure you have collected at the very least: Identifiers, Basic demographics, Clinican Diagnoses, TOWRE scores, and EHQ scores from the HBN data portal

### This could be adapted for other studies that use this suite of neuroimaging scripts on different subjects:
1) Make sure your phenotypic file is organized in the same way (one row per subject, one column per trait)
2) Either change the headings of your phenotypic file to match the HBN headers, or change the script to match your headers
3) Point the variable "HBN_query" to your phenotypic file
4) Change the "representative_subject" variable to one with good quality data and a complete gradient table

# 1) Import packages and load data
### Make sure to read the instructions in the github and the preamble to this notebook

In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import filecmp
import json
import scipy.stats
# CHANGE THIS VARIABLE TO REFLECT WHERE YOUR HBN DATA LIVE ()
bids_dir = '/' # Path should end with a '/'

# IF USING YOUR OWN DATASET, CHANGE THIS VARIABLE TO A SUBJECT WITH A GOOD QUALITY RUN AND COMPLETE GRADIENT TABLE
representative_subject = 'sub-NDARAA306NT2'
num_bad_slices_thresh = 9 # Number of bad slices to be excluded; Change this if you wish

# THESE VARIABLES SHOULD NOT CHANGE IF THE ANALYSIS WAS RUN ACCORDING TO INSTRUCTIONS
code_dir = bids_dir+'code/'
derivatives_dir = bids_dir+'derivatives/'
qsiprep_dir = derivatives_dir+'qsiprep/'
tractseg_dir = derivatives_dir+'TractSeg/'

# LOAD THE PHENOTYPIC DATA
HBN_query = pd.read_csv(code_dir+'HBN_query.csv')
# GET SUBJECTS WITH PHENOTYPIC DATA
query_subs = ['sub-'+name[:-11] for name in HBN_query['Identifiers']]
# GET SUBJECTS WITH SEGMENTATION DATA
tractseg_subs = np.transpose([path.split('/')[-1] for path in glob.glob(tractseg_dir+'sub-*')])
tractseg_subs.sort()

# 2) Find intersection of subjects with valid TOWRE scores, handedness data, and segmentation data

### This assumes any folder in your TractSeg directory beginning with "sub-" has complete data. Please remove failed TractSeg subjects from your TractSeg directory or rename them (e.g. "BAD_sub-XX")

In [ ]:
# GET SUBJECTS WITH VALID TOWRE SCORES, HANDEDNESS DATA, AND SES SCORES
towre_query_subs = np.asarray(query_subs)[HBN_query['TOWRE,TOWRE_Valid']=='1']
hand_query_subs = np.asarray(query_subs)[HBN_query['EHQ,Data_entry']=='Complete']
ses_subs = np.asarray(query_subs)[HBN_query['Barratt,Data_entry']=='Complete']

# GET INTERSECTION OF SUBJECTS WITH PHENOTYPIC AND SEGMENTATION DATA
subs_pheno_seg = list(set(tractseg_subs) & set(towre_query_subs) & set(hand_query_subs))
subs_pheno_seg.sort()
# FIND WHERE THESE SUBJECTS ARE LOCATED IN THE PHENOTYPIC FILE
pheno_seg_query_indexes = np.transpose([np.where(np.asarray(query_subs)==sub)[0][0] for sub in subs_pheno_seg])
# EXTRACT THE TOWRE SCORES
towre_scores_with_nan=HBN_query['TOWRE,TOWRE_Total_Scaled'][pheno_seg_query_indexes]
# FILTER OUT THE NANs
good_inds=np.asarray(towre_scores_with_nan.isnull()==False)
towre_scores = np.asarray(list(map(int, np.asarray(towre_scores_with_nan[good_inds]))))
# GET LIST OF SUBEJCTS WITH AND FULL PHENOTYPIC DATA AND SEGMENTATION DATA
subs_pheno_seg = np.asarray(subs_pheno_seg)[good_inds]
# GET THEIR LOCATIONS IN THE PHENOTYPIC FILE
pheno_seg_query_indexes = pheno_seg_query_indexes[good_inds]
print('A total of',np.size(subs_pheno_seg),'subjects have full phenotypic and segmentation data.')

# 3) Quality Control
### Here we use the QC outputs of QSIPrep to exclude any subjects with poor quality images, inconsistent diffusion parameters, or an age > 18.
### If you have additional subjects you would like to exclude (e.g. visually analyzing QSIPrep HTML files or TractSeg outputs), add their full subject name (e.g. sub-PROJECTXXXX) to the variable "more_subs_exclude"

In [ ]:
more_subs_exclude = []

# INITIALIZE OUTPUT VARIABLES
subs_qc = [] # sub names
inds_qc = [] # phenotypic file indexes

# LOAD QSIPREP GROUP QC OUTPUT
qc_file  = qsiprep_dir+'dwiqc.json'
with open(qc_file) as f:
        qc_data = json.load(f)['subjects']
# GET ORDER OF SUBJECTS in QC Data
qc_sub_order = np.asarray([sub_data['subject_id'] for sub_data in qc_data])
# LOAD REPRESENTATIVE B-VALUES TO COMPARE AGAINST
b_val = glob.glob(qsiprep_dir+representative_subject+'/dwi/*.bval')[0]

# RUN QC
print('SUBJECTS BELOW HAVE BEEN EXCLUDED, THE THREE CRITERIA ARE LISTED NEXT TO THE SUBJECT NAME')
for ind in range(np.size(subs_pheno_seg)):
    sub = subs_pheno_seg[ind]
    query_ind = pheno_seg_query_indexes[ind]
    # EXCLUDE SUBJECT IF MANUALLY ADDED TO EXCLUSION LIST
    if sub in more_subs_exclude:
        continue
        
    # COMPARE B-VALUES TO REPRESENTATIVE SUBJECT
    b_val_sub = glob.glob(qsiprep_dir+sub+'/dwi/*.bval')[0]
    b_val_comparison = filecmp.cmp(b_val,b_val_sub)
    # LOAD QC DATA AND CHECK NUMBER OF BAD SLICES
    qc_ind = [index for index in range(np.size(qc_sub_order)) if qc_sub_order[index] == sub][0]
    num_bad_slices = qc_data[qc_ind]['t1_num_bad_slices']
    # CHECK AGE
    age_check = np.asarray(HBN_query['Basic_Demos,Age'][query_ind]) < 19
    # IF QC PASSSES, SAVE OUT SUBJECT
    if num_bad_slices <= num_bad_slices_thresh and b_val_comparison and age_check:
        subs_qc.append(sub)
        inds_qc.append(pheno_seg_query_indexes[ind])
    # IF NOT, PRINT SUBJECT NAME AND WHY THEY DID NOT PASS
    else: 
        print(sub, 'num_bad_slices =',num_bad_slices, ', b_val_comparison =',b_val_comparison, ', age_check =',age_check)
        
# CONVERT TO A NUMPY ARRAY FOR CONVENIENCE
subs_qc = np.asarray(subs_qc)
inds_qc = np.asarray(inds_qc)

# 4) Load demographic data only for QC subjects
### To distinguish between typical and poor readers, we use the conventional diagnostic criteria of an age-standardized TOWRE score threshold of 85

In [ ]:
# GET PHENOTYPE DATA OF QCed SUBJECTS
subs = subs_qc
sex = np.asarray(HBN_query['Basic_Demos,Sex'][inds_qc])
age = np.asarray(HBN_query['Basic_Demos,Age'][inds_qc])
hand = np.asarray(HBN_query['EHQ,EHQ_Total'][inds_qc]).astype(float)
towre = np.asarray(HBN_query['TOWRE,TOWRE_Total_Scaled'][inds_qc]).astype(float)
swe = np.asarray(HBN_query['TOWRE,TOWRE_SWE_Scaled'][inds_qc]).astype(float)
pde = np.asarray(HBN_query['TOWRE,TOWRE_PDE_Scaled'][inds_qc]).astype(float)
n = np.size(subs_qc)

# GET CLINICIAN DIAGNOSES
inds_td = []
inds_rd = []
for index in range(n):
    ind_query = inds_qc[index]
    dxs = [HBN_query[key][ind_query] for key in HBN_query.keys() if 'DX' in key]
    dx_check = [('Impairment in Reading' in dx) for dx in dxs if type(dx)==str]
    if sum(dx_check)>0:
        inds_rd.append(True)
        inds_td.append(False)         
    else:
        inds_td.append(True)
        inds_rd.append(False)

inds_td = np.asarray(inds_td)
inds_rd = np.asarray(inds_rd)

# SPLIT INTO HIGH AND LOW T
towre_thresh = 85 # Conventional diagnostic criterion
inds_high_t = (towre >= towre_thresh)
inds_low_t = (towre < towre_thresh)

# SEPARATE PHENOTYPIC INFO BY GROUP
subs_td = subs_qc[inds_td]
sex_td = sex[inds_td]
age_td = age[inds_td]
hand_td = hand[inds_td]
towre_td = towre[inds_td]
swe_td = swe[inds_td]
pde_td = pde[inds_td]
n_td = np.size(subs_td)

subs_rd = subs_qc[inds_rd]
sex_rd = sex[inds_rd]
age_rd = age[inds_rd]
hand_rd = hand[inds_rd]
towre_rd = towre[inds_rd]
swe_rd = swe[inds_rd]
pde_rd = pde[inds_rd]
n_rd = np.size(subs_rd)

subs_high_t = subs_qc[inds_high_t]
sex_high_t = sex[inds_high_t]
age_high_t = age[inds_high_t]
hand_high_t = hand[inds_high_t]
towre_high_t = towre[inds_high_t]
swe_high_t = swe[inds_high_t]
pde_high_t = pde[inds_high_t]
n_high_t = np.size(subs_high_t)

subs_low_t = subs_qc[inds_low_t]
sex_low_t = sex[inds_low_t]
age_low_t = age[inds_low_t]
hand_low_t = hand[inds_low_t]
towre_low_t = towre[inds_low_t]
swe_low_t = swe[inds_low_t]
pde_low_t = pde[inds_low_t]
n_low_t = np.size(subs_low_t)

#### Check for group differences (Table 1)

In [ ]:
# DIFFERENCES IN SEX DISTRIBUTION (Chi-Square)
f_td = sum(sex_td==1)
m_td = sum(sex_td==0)
f_rd = sum(sex_rd==1)
m_rd = sum(sex_rd==0)
f_high_t = sum(sex_high_t==1)
m_high_t = sum(sex_high_t==0)
f_low_t = sum(sex_low_t==1)
m_low_t = sum(sex_low_t==0)

print('AVERGAE AGE:',np.mean(age),'(',np.std(age),')')
print('AVERGAE HAND:',np.mean(hand),'(',np.std(hand),')')

# DIFFERENCES IN SEX (Chi-Square)
print('M TD:',m_td,', F TD:',f_td,', M RD',m_rd,', F RD',f_rd)
print('SEX: '+str(scipy.stats.chisquare([[m_td,m_rd],[f_td,f_rd]])),'\n')
# DIFFERENCES IN AGE (Welch's t-test)
print('AGE: '+str(scipy.stats.ttest_ind(age_td, age_rd, equal_var=False)))
print('TD MEAN (SD):',np.mean(age_td),'(',np.std(age_td),'), RD MEAN (SD):', np.mean(age_rd),'(',np.std(age_rd),')\n')
# DIFFERENCES IN HANDEDNESS (Welch's t-test)
print('HAND: '+str(scipy.stats.ttest_ind(hand_td, hand_rd, equal_var=False)))
print('TD MEAN (SD):',np.mean(hand_td),'(',np.std(hand_td),'), RD MEAN (SD):', np.mean(hand_rd),'(',np.std(hand_rd),')\n')
# DIFFERENCES IN SEX (Chi-Square)
print('M HIGH T:',m_high_t,', F HIGH T:',f_high_t,', M LOW T',m_low_t,', F LOW T',f_low_t)
print('SEX: '+str(scipy.stats.chisquare([[m_td,m_low_t],[f_td,f_low_t]])),'\n')
# DIFFERENCES IN AGE (Welch's t-test)
print('AGE: '+str(scipy.stats.ttest_ind(age_high_t, age_low_t, equal_var=False)))
print('HIGH T MEAN (SD):',np.mean(age_high_t),'(',np.std(age_high_t),'), LOW T MEAN (SD):', np.mean(age_low_t),'(',np.std(age_low_t),')\n')
# DIFFERENCES IN HANDEDNESS (Welch's t-test)
print('HAND: '+str(scipy.stats.ttest_ind(hand_high_t, hand_rd, equal_var=False)))
print('HIGH T MEAN (SD):',np.mean(hand_high_t),'(',np.std(hand_high_t),'), LOW T MEAN (SD):', np.mean(hand_low_t),'(',np.std(hand_low_t),')\n')

#### Figure 1: TOWRE Score distributions

In [ ]:
plt.figure(figsize=(10,10))
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
plt.vlines(85,0,43.5,'k',linewidth=5,linestyle=':')
plt.rc('font', **font)
plt.hist(towre_td,bins=50,color='gray')
plt.hist(towre_rd,bins=50,color='black')

plt.text(55.5,40,('N(RD) = '+str(n_rd)))
plt.text(120,40,('N(TD) = '+str(n_td)))
plt.text(50,38,('N(T<85) = '+str(n_low_t)))
plt.text(114.5,38,('N(T≥85) = '+str(n_high_t)))
plt.ylim([0,44])
plt.xlabel('TOWRE Total Score, Standardized',fontsize=35,fontweight='bold')
plt.ylabel('Number of Participants',fontsize=40,fontweight='bold')
#plt.savefig('TOWRE.pdf')
plt.show()



# 5) Create TractSeg Models
### Run this code block to create the text that goes into the bottom of the file fed into TractSeg.
#### First item has to be subject_ID, followed by either the correlation target (e.g. TOWRE score) or group ID (0 or 1). The next items are traits you want to regress out.
#### You can comment individual blocks of this to run premade models
#### The output of each model should be pasted into the TractSeg model .txt before generating plots

In [ ]:
# AGE TD
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_td,towre_td,sex_td,hand_td,age_td):
#    print(sub,age_sub,sex_sub,hand_sub)

# AGE RD
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_rd,towre_rd,sex_rd,hand_rd,age_rd):
#    print(sub,age_sub,sex_sub,hand_sub)

# AGE All
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs,towre,sex,hand,age):
#    print(sub,age_sub,sex_sub,hand_sub)

# TOWRE TD
#for sub,score,sex_sub,hand_sub in zip(subs_td,towre_td,sex_td,hand_td):
#    print(sub,score,sex_sub,hand_sub)
    
# TOWRE RD
#for sub,score,sex_sub,hand_sub in zip(subs_rd,towre_rd,sex_rd,hand_rd):
#    print(sub,score,sex_sub,hand_sub)
    
# TOWRE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,towre,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# SWE TD
#for sub,score,sex_sub,hand_sub in zip(subs_td,swe_td,sex_td,hand_td):
#    print(sub,score,sex_sub,hand_sub)
    
# SWE RD
#for sub,score,sex_sub,hand_sub in zip(subs_rd,swe_rd,sex_rd,hand_rd):
#    print(sub,score,sex_sub,hand_sub)
    
# SWE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,swe,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# PDE TD
#for sub,score,sex_sub,hand_sub in zip(subs_td,pde_td,sex_td,hand_td):
#    print(sub,score,sex_sub,hand_sub)
    
# PDE RD
#for sub,score,sex_sub,hand_sub in zip(subs_rd,pde_rd,sex_rd,hand_rd):
#    print(sub,score,sex_sub,hand_sub)
    
# PDE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,pde,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# GROUP RD VS TD
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_qc,towre,sex,hand,age):#
#    if sub in subs_td:
#        group = 0
#    else: group = 1
#    print(sub,group,sex_sub,hand_sub,age_sub)


In [ ]:
# AGE high_t
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_high_t,towre_high_t,sex_high_t,hand_high_t,age_high_t):
#    print(sub,age_sub,sex_sub,hand_sub)

# AGE low_t
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_low_t,towre_low_t,sex_low_t,hand_low_t,age_low_t):
#    print(sub,age_sub,sex_sub,hand_sub)

# AGE All
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs,towre,sex,hand,age):
#    print(sub,age_sub,sex_sub,hand_sub)

# TOWRE high_t
#for sub,score,sex_sub,hand_sub in zip(subs_high_t,towre_high_t,sex_high_t,hand_high_t):
#    print(sub,score,sex_sub,hand_sub)
    
# TOWRE low_t
#for sub,score,sex_sub,hand_sub in zip(subs_low_t,towre_low_t,sex_low_t,hand_low_t):
#    print(sub,score,sex_sub,hand_sub)
    
# TOWRE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,towre,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# SWE high_t
#for sub,score,sex_sub,hand_sub in zip(subs_high_t,swe_high_t,sex_high_t,hand_high_t):
#    print(sub,score,sex_sub,hand_sub)
    
# SWE low_t
# sub,score,sex_sub,hand_sub in zip(subs_low_t,swe_low_t,sex_low_t,hand_low_t):
#    print(sub,score,sex_sub,hand_sub)
    
# SWE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,swe,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# PDE high_t
#for sub,score,sex_sub,hand_sub in zip(subs_high_t,pde_high_t,sex_high_t,hand_high_t):
#    print(sub,score,sex_sub,hand_sub)
    
# PDE low_t
#for sub,score,sex_sub,hand_sub in zip(subs_low_t,pde_low_t,sex_low_t,hand_low_t):
#    print(sub,score,sex_sub,hand_sub)
    
# PDE ALL
#for sub,score,sex_sub,hand_sub in zip(subs,pde,sex,hand):
#    print(sub,score,sex_sub,hand_sub)

# GROUP low_t VS high_t
#for sub,towre_sub,sex_sub,hand_sub,age_sub in zip(subs_qc,towre,sex,hand,age):#
#    if sub in subs_high_t:
#        group = 0
#    else: group = 1
#    print(sub,group,sex_sub,hand_sub,age_sub)
